In [127]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
import scipy.stats as st

# from textwrap import wrap

%matplotlib inline

# %run cleaning_join_all.ipynb
# df_join_clean = import_df_join_clean() # type: ignore

df_join_clean = pd.read_csv("../data/clean/df_join_clean.csv")

In [128]:
display(df_join_clean)

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,age_group,bal_category_3,bal_category_2,Variation
0,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:27:07,5,64,79,U,2,189023.86,1,4,Elderly,High,High,Test
1,9988021,580560515_7732621733,781255054_21935453173_531117,2,2017-04-17 15:26:51,5,64,79,U,2,189023.86,1,4,Elderly,High,High,Test
2,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:19:22,5,64,79,U,2,189023.86,1,4,Elderly,High,High,Test
3,9988021,580560515_7732621733,781255054_21935453173_531117,2,2017-04-17 15:19:13,5,64,79,U,2,189023.86,1,4,Elderly,High,High,Test
4,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:18:04,5,64,79,U,2,189023.86,1,4,Elderly,High,High,Test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319497,1574008,117364417_77840596075,528720790_71583064618_169151,0,2017-05-06 23:43:27,10,121,55,U,2,153238.83,3,6,Senior,High,High,Test
319498,2908510,814969699_90652851448,562606085_36368381773_92090,0,2017-05-10 22:57:17,21,252,34,M,3,141808.05,6,9,Adults,High,High,Control
319499,2908510,814969699_90652851448,562606085_36368381773_92090,2,2017-05-10 22:56:31,21,252,34,M,3,141808.05,6,9,Adults,High,High,Control
319500,2908510,814969699_90652851448,562606085_36368381773_92090,1,2017-05-10 22:56:23,21,252,34,M,3,141808.05,6,9,Adults,High,High,Control


In [ ]:
df_join_clean = df_join_clean.sort_values(by=["client_id", "visit_id", "date_time", "process_step" ]).reset_index(drop=True)
df_join_clean

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,age_group,bal_category_3,bal_category_2,Variation
0,555,402506806_56087378777,637149525_38041617439_716659,0,2017-04-15 12:57:56,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test
1,555,402506806_56087378777,637149525_38041617439_716659,1,2017-04-15 12:58:03,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test
2,555,402506806_56087378777,637149525_38041617439_716659,2,2017-04-15 12:58:35,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test
3,555,402506806_56087378777,637149525_38041617439_716659,3,2017-04-15 13:00:14,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test
4,555,402506806_56087378777,637149525_38041617439_716659,4,2017-04-15 13:00:34,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319497,9999729,834634258_21862004160,870243567_56915814033_814203,4,2017-05-08 16:09:40,10,124,31,F,3,107059.74,6,9,Adults,High,High,Test
319498,9999729,604429154_69247391147,99583652_41711450505_426179,0,2017-04-05 13:40:49,10,124,31,F,3,107059.74,6,9,Adults,High,High,Test
319499,9999729,604429154_69247391147,99583652_41711450505_426179,1,2017-04-05 13:41:04,10,124,31,F,3,107059.74,6,9,Adults,High,High,Test
319500,9999832,145538019_54444341400,472154369_16714624241_585315,0,2017-05-16 16:46:03,23,281,49,F,2,431887.61,1,4,Adults,High,High,Test


In [130]:
df_join_clean[df_join_clean["process_step"] == 4].drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate confirms for same visit
df_join_clean[df_join_clean["process_step"] == 0].drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate starts for same visit

/var/folders/x5/xrbwdz1510vcx8w5dcdp2s7m0000gn/T/ipykernel_92048/301666466.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_join_clean[df_join_clean["process_step"] == 4].drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate confirms for same visit
/var/folders/x5/xrbwdz1510vcx8w5dcdp2s7m0000gn/T/ipykernel_92048/301666466.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_join_clean[df_join_clean["process_step"] == 0].drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate starts for same visit


## Considerations after combining the data:

In [ ]:
# Eveluates funnel conversion

df_join_clean['steps_status'] = None
previous_visit_id = None
previous_value = None

for i in range(len(df_join_clean)):
    current_visit_id = df_join_clean.iloc[i]["visit_id"]
    current_value = df_join_clean.iloc[i]["process_step"]

    if current_value == 0:
        df_join_clean.iloc[i, df_join_clean.columns.get_loc('steps_status')] = 0
    else:
        if previous_value > current_value:
            df_join_clean.iloc[i, df_join_clean.columns.get_loc('steps_status')] = -1
        if previous_value == current_value:
            df_join_clean.iloc[i, df_join_clean.columns.get_loc('steps_status')] = 0
        if previous_value < current_value:
            df_join_clean.iloc[i, df_join_clean.columns.get_loc('steps_status')] = 1
        # if current_value == 4:
        #     df_join_clean.iloc[i, df_join_clean.columns.get_loc('steps_status')] = "confirm"
    
    previous_value = current_value

print(df_join_clean)

        client_id             visitor_id                      visit_id  \
0             555  402506806_56087378777  637149525_38041617439_716659   
1             555  402506806_56087378777  637149525_38041617439_716659   
2             555  402506806_56087378777  637149525_38041617439_716659   
3             555  402506806_56087378777  637149525_38041617439_716659   
4             555  402506806_56087378777  637149525_38041617439_716659   
...           ...                    ...                           ...   
319497    9999729  834634258_21862004160  870243567_56915814033_814203   
319498    9999729  604429154_69247391147   99583652_41711450505_426179   
319499    9999729  604429154_69247391147   99583652_41711450505_426179   
319500    9999832  145538019_54444341400  472154369_16714624241_585315   
319501    9999832  145538019_54444341400  472154369_16714624241_585315   

        process_step            date_time  clnt_tenure_yr  clnt_tenure_mnth  \
0                  0  2017-04-15

In [132]:
# Eveluates if steps are errors

df_join_clean['step_error'] = None
previous_visit_id = None
previous_value = None

for i in range(len(df_join_clean)):
    current_visit_id = df_join_clean.iloc[i]["visit_id"]
    current_value = df_join_clean.iloc[i]["process_step"]

    if current_value == 0:
        df_join_clean.iloc[i, df_join_clean.columns.get_loc('step_error')] = int(0)
    else:
        if previous_value > current_value:
            df_join_clean.iloc[i, df_join_clean.columns.get_loc('step_error')] = int(1)
        if previous_value == current_value:
            df_join_clean.iloc[i, df_join_clean.columns.get_loc('step_error')] = int(0)
        if previous_value < current_value:
            df_join_clean.iloc[i, df_join_clean.columns.get_loc('step_error')] = int(0)
        # if current_value == 4:
        #     df_join_clean.iloc[i, df_join_clean.columns.get_loc('step_error')] = "confirm"
    
    previous_value = current_value

print(df_join_clean)

        client_id             visitor_id                      visit_id  \
0             555  402506806_56087378777  637149525_38041617439_716659   
1             555  402506806_56087378777  637149525_38041617439_716659   
2             555  402506806_56087378777  637149525_38041617439_716659   
3             555  402506806_56087378777  637149525_38041617439_716659   
4             555  402506806_56087378777  637149525_38041617439_716659   
...           ...                    ...                           ...   
319497    9999729  834634258_21862004160  870243567_56915814033_814203   
319498    9999729  604429154_69247391147   99583652_41711450505_426179   
319499    9999729  604429154_69247391147   99583652_41711450505_426179   
319500    9999832  145538019_54444341400  472154369_16714624241_585315   
319501    9999832  145538019_54444341400  472154369_16714624241_585315   

        process_step            date_time  clnt_tenure_yr  clnt_tenure_mnth  \
0                  0  2017-04-15

In [133]:
# Cast floats as integers
df_join_clean = df_join_clean.astype({
    "steps_status": "int64", 
    "step_error": "int64"
    })

df_join_clean

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,age_group,bal_category_3,bal_category_2,Variation,steps_status,step_error
0,555,402506806_56087378777,637149525_38041617439_716659,0,2017-04-15 12:57:56,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test,0,0
1,555,402506806_56087378777,637149525_38041617439_716659,1,2017-04-15 12:58:03,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test,1,0
2,555,402506806_56087378777,637149525_38041617439_716659,2,2017-04-15 12:58:35,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test,1,0
3,555,402506806_56087378777,637149525_38041617439_716659,3,2017-04-15 13:00:14,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test,1,0
4,555,402506806_56087378777,637149525_38041617439_716659,4,2017-04-15 13:00:34,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319497,9999729,834634258_21862004160,870243567_56915814033_814203,4,2017-05-08 16:09:40,10,124,31,F,3,107059.74,6,9,Adults,High,High,Test,1,0
319498,9999729,604429154_69247391147,99583652_41711450505_426179,0,2017-04-05 13:40:49,10,124,31,F,3,107059.74,6,9,Adults,High,High,Test,0,0
319499,9999729,604429154_69247391147,99583652_41711450505_426179,1,2017-04-05 13:41:04,10,124,31,F,3,107059.74,6,9,Adults,High,High,Test,1,0
319500,9999832,145538019_54444341400,472154369_16714624241_585315,0,2017-05-16 16:46:03,23,281,49,F,2,431887.61,1,4,Adults,High,High,Test,0,0


### Split the data in the control and test groups

In [146]:
df_test_group = df_join_clean[df_join_clean["Variation"] == "Test"]
df_control_group = df_join_clean[df_join_clean["Variation"] == "Control"]

display(df_test_group)
display(df_control_group)

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,age_group,bal_category_3,bal_category_2,Variation,steps_status,step_error
0,555,402506806_56087378777,637149525_38041617439_716659,0,2017-04-15 12:57:56,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test,0,0
1,555,402506806_56087378777,637149525_38041617439_716659,1,2017-04-15 12:58:03,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test,1,0
2,555,402506806_56087378777,637149525_38041617439_716659,2,2017-04-15 12:58:35,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test,1,0
3,555,402506806_56087378777,637149525_38041617439_716659,3,2017-04-15 13:00:14,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test,1,0
4,555,402506806_56087378777,637149525_38041617439_716659,4,2017-04-15 13:00:34,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319497,9999729,834634258_21862004160,870243567_56915814033_814203,4,2017-05-08 16:09:40,10,124,31,F,3,107059.74,6,9,Adults,High,High,Test,1,0
319498,9999729,604429154_69247391147,99583652_41711450505_426179,0,2017-04-05 13:40:49,10,124,31,F,3,107059.74,6,9,Adults,High,High,Test,0,0
319499,9999729,604429154_69247391147,99583652_41711450505_426179,1,2017-04-05 13:41:04,10,124,31,F,3,107059.74,6,9,Adults,High,High,Test,1,0
319500,9999832,145538019_54444341400,472154369_16714624241_585315,0,2017-05-16 16:46:03,23,281,49,F,2,431887.61,1,4,Adults,High,High,Test,0,0


,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,age_group,bal_category_3,bal_category_2,Variation,steps_status,step_error
14,1028,42237450_62128060588,557292053_87239438319_391157,0,2017-04-08 18:51:28,12,145,36,M,3,103520.22,1,4,Adults,High,High,Control,0,0
15,1028,42237450_62128060588,557292053_87239438319_391157,1,2017-04-08 18:52:17,12,145,36,M,3,103520.22,1,4,Adults,High,High,Control,1,0
16,1028,42237450_62128060588,557292053_87239438319_391157,1,2017-04-08 18:53:20,12,145,36,M,3,103520.22,1,4,Adults,High,High,Control,0,0
17,1028,42237450_62128060588,557292053_87239438319_391157,2,2017-04-08 18:53:29,12,145,36,M,3,103520.22,1,4,Adults,High,High,Control,1,0
18,1028,42237450_62128060588,557292053_87239438319_391157,3,2017-04-08 18:58:04,12,145,36,M,3,103520.22,1,4,Adults,High,High,Control,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319454,9998346,292425655_16607136645,189177304_69869411700_783154,3,2017-03-29 15:30:25,50,602,62,F,2,149881.38,6,9,Senior,High,High,Control,1,0
319455,9998346,292425655_16607136645,189177304_69869411700_783154,3,2017-03-29 15:37:28,50,602,62,F,2,149881.38,6,9,Senior,High,High,Control,0,0
319456,9998346,292425655_16607136645,189177304_69869411700_783154,3,2017-03-29 15:37:28,50,602,62,F,2,149881.38,6,9,Senior,High,High,Control,0,0
319457,9998346,292425655_16607136645,189177304_69869411700_783154,4,2017-03-29 15:37:28,50,602,62,F,2,149881.38,6,9,Senior,High,High,Control,1,0


### Sort the datasets by: client_id, visit_id (one customer can use the platform several times), process, and datetime


In [135]:
# # Required:
# # - change process_step to numeral (0 to 4)
# # - evaluate which floats to change to integers
# df_test_group = df_test_group.sort_values(by=["client_id", "visit_id", "process_step", "date_time"]).reset_index(drop=True)
# df_control_group = df_control_group.sort_values(by=["client_id", "visit_id", "process_step", "date_time"]).reset_index(drop=True)

In [152]:
# Check random client_ids test group

list_of_ids = df_test_group["client_id"].to_list()
df_test_group[df_test_group["client_id"] == random.choice(list_of_ids)][["client_id", "visit_id",  "date_time", "process_step", "steps_status", "step_error"]]

,client_id,visit_id,date_time,process_step,steps_status,step_error
286930,8958079,877660092_24243832181_99983,2017-06-17 15:42:01,0,0,0
286931,8958079,877660092_24243832181_99983,2017-06-17 15:43:35,1,1,0
286932,8958079,877660092_24243832181_99983,2017-06-17 15:44:09,2,1,0
286933,8958079,877660092_24243832181_99983,2017-06-17 15:44:44,1,-1,1
286934,8958079,877660092_24243832181_99983,2017-06-17 15:44:59,0,0,0
286935,8958079,956869118_59654915472_994422,2017-05-24 15:39:36,0,0,0
286936,8958079,956869118_59654915472_994422,2017-05-24 15:45:19,0,0,0
286937,8958079,956869118_59654915472_994422,2017-05-24 15:45:52,0,0,0


In [137]:
# Spot Errors
df_test_group[df_test_group["steps_status"] == -1]

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,age_group,bal_category_3,bal_category_2,Variation,steps_status,step_error
60,1516,182314299_63168583136,481123290_95510977345_707323,1,2017-04-05 15:55:23,12,150,59,F,2,25408.39,5,8,Senior,Low,Low,Test,-1,1
67,1516,182314299_63168583136,481123290_95510977345_707323,1,2017-04-05 15:57:36,12,150,59,F,2,25408.39,5,8,Senior,Low,Low,Test,-1,1
109,1936,883992205_24100452548,943469853_76229226125_839996,2,2017-03-29 12:55:28,5,63,27,M,2,52447.90,6,9,Young Adult,Medium,Low,Test,-1,1
300,9751,371374112_82466346974,158586948_58770146204_677956,1,2017-05-29 08:06:44,5,63,36,M,2,30597.08,4,7,Adults,Low,Low,Test,-1,1
430,15753,601984694_38256669083,899381947_58477487045_97772,2,2017-06-08 12:47:25,6,75,58,M,2,59565.00,3,6,Senior,Medium,Low,Test,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319331,9994764,208048562_5591648325,945115982_5995160722_563889,2,2017-04-24 12:22:18,5,64,31,U,2,58258.46,6,9,Adults,Medium,Low,Test,-1,1
319332,9994764,208048562_5591648325,945115982_5995160722_563889,1,2017-04-24 12:22:19,5,64,31,U,2,58258.46,6,9,Adults,Medium,Low,Test,-1,1
319364,9995295,719082482_48238561745,944432037_72425410366_517931,1,2017-03-29 11:17:07,33,397,58,U,2,166663.77,2,5,Senior,High,High,Test,-1,1
319367,9995295,719082482_48238561745,944432037_72425410366_517931,1,2017-03-29 11:18:50,33,397,58,U,2,166663.77,2,5,Senior,High,High,Test,-1,1


In [138]:
df_control_group.dtypes

client_id             int64
visitor_id           object
visit_id             object
process_step          int64
date_time            object
clnt_tenure_yr        int64
clnt_tenure_mnth      int64
clnt_age              int64
gendr                object
num_accts             int64
bal                 float64
calls_6_mnth          int64
logons_6_mnth         int64
age_group            object
bal_category_3       object
bal_category_2       object
Variation            object
steps_status          int64
step_error            int64
dtype: object

In [ ]:
# Completion Rate by Group

def get_completion_rate(df):
    num_visits = df["visit_id"].nunique()
    print("Number of Visits >> ", num_visits)

    num_confirm = df[df["process_step"] == 4]
    num_confirm.drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate confirms for same visit
    num_confirm = num_confirm["process_step"].count()
    print("Number of Confirms >> ", num_confirm)

    completion_rate = (num_confirm / num_visits).round(2)
    print("Completion Rate >> ", completion_rate)


Test Group
Number of Visits >>  36906
Number of Confirms >>  21583
Completion Rate >>  0.58

Test Group bal > 1000000
Number of Visits >>  753
Number of Confirms >>  477
Completion Rate >>  0.63

Test Group bal > 1000000 and Male
Number of Visits >>  499
Number of Confirms >>  318
Completion Rate >>  0.64

Test Group bal > 1000000 and Adult Males
Number of Visits >>  64
Number of Confirms >>  43
Completion Rate >>  0.67



/var/folders/x5/xrbwdz1510vcx8w5dcdp2s7m0000gn/T/ipykernel_92048/1070585648.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_confirm.drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate confirms for same visit


In [154]:
print("Test Group")
get_completion_rate(df_test_group)
print("")

print("Test Group bal > 1000000")
get_completion_rate(df_test_group[df_test_group["bal"] > 1000000])
print("")

print("Test Group bal > 1000000 and Male")
get_completion_rate(df_test_group[(df_test_group["bal"] > 1000000) & (df_test_group["gendr"] == "M")])
print("")

print("Test Group bal > 1000000 and Adult Males")
get_completion_rate(df_test_group[(df_test_group["bal"] > 1000000) & (df_test_group["gendr"] == "M") & (df_test_group["age_group"] == "Adults") ])
print("")

Test Group
Number of Visits >>  36906
Number of Confirms >>  21583
Completion Rate >>  0.58

Test Group bal > 1000000
Number of Visits >>  753
Number of Confirms >>  477
Completion Rate >>  0.63

Test Group bal > 1000000 and Male
Number of Visits >>  499
Number of Confirms >>  318
Completion Rate >>  0.64

Test Group bal > 1000000 and Adult Males
Number of Visits >>  64
Number of Confirms >>  43
Completion Rate >>  0.67



/var/folders/x5/xrbwdz1510vcx8w5dcdp2s7m0000gn/T/ipykernel_92048/1070585648.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_confirm.drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate confirms for same visit


In [155]:
print("Control Group")
get_completion_rate(df_control_group)
print("")

print("Control Group bal > 1000000")
get_completion_rate(df_control_group[df_control_group["bal"] > 1000000])
print("")

print("Control Group bal > 1000000 and Male")
get_completion_rate(df_control_group[(df_control_group["bal"] > 1000000) & (df_control_group["gendr"] == "M")])
print("")

print("Control Group bal > 1000000 and Adult Males")
get_completion_rate(df_control_group[(df_control_group["bal"] > 1000000) & (df_control_group["gendr"] == "M") & (df_control_group["age_group"] == "Adults") ])
print("")

Control Group
Number of Visits >>  32020
Number of Confirms >>  15948
Completion Rate >>  0.5

Control Group bal > 1000000
Number of Visits >>  639
Number of Confirms >>  307
Completion Rate >>  0.48

Control Group bal > 1000000 and Male
Number of Visits >>  466
Number of Confirms >>  226
Completion Rate >>  0.48

Control Group bal > 1000000 and Adult Males
Number of Visits >>  67
Number of Confirms >>  40
Completion Rate >>  0.6



/var/folders/x5/xrbwdz1510vcx8w5dcdp2s7m0000gn/T/ipykernel_92048/1070585648.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_confirm.drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate confirms for same visit


### Determine which percentage of customers in any step of process are hitting errors for control, and test groups.
- Some clients will hit errors in any step of the process and they will go to a previous step. 

In [156]:
# Error Rate by Group

def get_error_rate(df):
    num_visits = df["visit_id"].nunique()
    print("Number of Visits >> ", num_visits)

    num_steps = df["process_step"].count()
    print("Number of Steps >> ", num_steps)

    num_errors = df[df["step_error"] == 1]
    # num_errors.drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate confirms for same visit
    num_errors = num_errors["steps_status"].count()
    print("Number of Errors >> ", num_errors)

    error_rate = (num_errors / num_visits).round(2)
    print("Error Rate per Visit >> ", error_rate)

    error_rate_step = (num_errors / num_steps).round(2)
    print("Error Rate per Steps >> ", error_rate_step)

print("Test Group")
get_error_rate(df_test_group)
print("")

print("Control Group")
get_error_rate(df_control_group)
print("")





Test Group
Number of Visits >>  36906
Number of Steps >>  176807
Number of Errors >>  5953
Error Rate per Visit >>  0.16
Error Rate per Steps >>  0.03

Control Group
Number of Visits >>  32020
Number of Steps >>  142695
Number of Errors >>  4887
Error Rate per Visit >>  0.15
Error Rate per Steps >>  0.03



In [157]:
print("Test Group bal > 1000000")
get_error_rate(df_test_group[df_test_group["bal"] > 1000000])
print("")

print("Control Group bal > 1000000")
get_error_rate(df_control_group[df_control_group["bal"] > 1000000])
print("")

Test Group bal > 1000000
Number of Visits >>  753
Number of Steps >>  3420
Number of Errors >>  131
Error Rate per Visit >>  0.17
Error Rate per Steps >>  0.04

Control Group bal > 1000000
Number of Visits >>  639
Number of Steps >>  2777
Number of Errors >>  74
Error Rate per Visit >>  0.12
Error Rate per Steps >>  0.03



In [158]:
print("Test Group bal > 1000000 and Male")
get_error_rate(df_test_group[(df_test_group["bal"] > 1000000) & (df_test_group["gendr"] == "M")])
print("")

print("Control Group bal > 1000000 and Male")
get_error_rate(df_control_group[(df_control_group["bal"] > 1000000) & (df_control_group["gendr"] == "M")])
print("")

Test Group bal > 1000000 and Male
Number of Visits >>  499
Number of Steps >>  2323
Number of Errors >>  98
Error Rate per Visit >>  0.2
Error Rate per Steps >>  0.04

Control Group bal > 1000000 and Male
Number of Visits >>  466
Number of Steps >>  1919
Number of Errors >>  54
Error Rate per Visit >>  0.12
Error Rate per Steps >>  0.03



In [ ]:
df_test_group
# df_test_group[df_control_group["age_group"] == "Adults"]

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,age_group,bal_category_3,bal_category_2,Variation,steps_status,step_error
0,555,402506806_56087378777,637149525_38041617439_716659,0,2017-04-15 12:57:56,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test,0,0
1,555,402506806_56087378777,637149525_38041617439_716659,1,2017-04-15 12:58:03,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test,1,0
2,555,402506806_56087378777,637149525_38041617439_716659,2,2017-04-15 12:58:35,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test,1,0
3,555,402506806_56087378777,637149525_38041617439_716659,3,2017-04-15 13:00:14,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test,1,0
4,555,402506806_56087378777,637149525_38041617439_716659,4,2017-04-15 13:00:34,3,46,30,U,2,25454.66,2,6,Young Adult,Low,Low,Test,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319497,9999729,834634258_21862004160,870243567_56915814033_814203,4,2017-05-08 16:09:40,10,124,31,F,3,107059.74,6,9,Adults,High,High,Test,1,0
319498,9999729,604429154_69247391147,99583652_41711450505_426179,0,2017-04-05 13:40:49,10,124,31,F,3,107059.74,6,9,Adults,High,High,Test,0,0
319499,9999729,604429154_69247391147,99583652_41711450505_426179,1,2017-04-05 13:41:04,10,124,31,F,3,107059.74,6,9,Adults,High,High,Test,1,0
319500,9999832,145538019_54444341400,472154369_16714624241_585315,0,2017-05-16 16:46:03,23,281,49,F,2,431887.61,1,4,Adults,High,High,Test,0,0


In [187]:
print("Test Group bal > 1000000 and Adult Male")
get_error_rate(df_test_group[(df_test_group["bal"] > 1000000) & (df_test_group["gendr"] == "M") & (df_test_group["age_group"] == "Adults")])
print("")

print("Control Group bal > 1000000 and Adult Male")
get_error_rate(df_control_group[(df_control_group["bal"] > 1000000) & (df_control_group["gendr"] == "M") & (df_control_group["age_group"] == "Adults")])
print("")


Test Group bal > 1000000 and Adult Male
Number of Visits >>  64
Number of Steps >>  301
Number of Errors >>  9
Error Rate per Visit >>  0.14
Error Rate per Steps >>  0.03

Control Group bal > 1000000 and Adult Male
Number of Visits >>  67
Number of Steps >>  270
Number of Errors >>  6
Error Rate per Visit >>  0.09
Error Rate per Steps >>  0.02



In [189]:

print("Control Group Young Adult")
get_error_rate(df_control_group[(df_control_group["age_group"] == "Young Adult")])
print("")

print("Control Group Adult")
get_error_rate(df_control_group[(df_control_group["age_group"] == "Adults")])
print("")

print("Control Group Senior")
get_error_rate(df_control_group[(df_control_group["age_group"] == "Senior")])
print("")

print("Control Group Elderly")
get_error_rate(df_control_group[(df_control_group["age_group"] == "Elderly")])
print("")


Control Group Young Adult
Number of Visits >>  5161
Number of Steps >>  23458
Number of Errors >>  810
Error Rate per Visit >>  0.16
Error Rate per Steps >>  0.03

Control Group Adult
Number of Visits >>  10897
Number of Steps >>  49128
Number of Errors >>  1506
Error Rate per Visit >>  0.14
Error Rate per Steps >>  0.03

Control Group Senior
Number of Visits >>  13449
Number of Steps >>  59989
Number of Errors >>  2214
Error Rate per Visit >>  0.16
Error Rate per Steps >>  0.04

Control Group Elderly
Number of Visits >>  2520
Number of Steps >>  10120
Number of Errors >>  357
Error Rate per Visit >>  0.14
Error Rate per Steps >>  0.04




- If a customer has several start times, only the last one must be considered.
- If a customer hits the confirmation step several times, consider the last one as the end of the process
- Keep in mind that this is a funnel process. Therefore, not all the customers who start the process will finish it and be able to proceed to the next one.
- There are lots of customers who participated in the experiment over a limited period of time. Some of them will be faster or more successful than others, but all of them count. Therefore, we're looking for "mean" times or rates!!!


## Hypothesis Tests

In [145]:
error_test = df_test_group['step_error']
error_control = df_control_group['step_error']

#The test group error rate is lower on average than the control group.

alpha = 0.05

#H0: test_error_rate >= control_error_rate
#H1: test_error_rate < control_error_rate

stat, p_value = st.ttest_ind(error_test, error_control, equal_var=False, alternative='less')

if p_value > alpha:
    print(f"We fail to reject the null hypothesis (H0). There is no significant difference in average tenure between the test and control groups (p-value: {p_value}).")
else:
    print(f"We reject the null hypothesis (H0). There is a significant difference in average tenure between the test and control groups (p-value: {p_value}).")

We fail to reject the null hypothesis (H0). There is no significant difference in average tenure between the test and control groups (p-value: 0.18487022365463246).




## Accordingly:

### Completion Rates
- Within the same group (control or test), are the completion rates (number of customers who reached the end of a step out of total who started) in each step the same? (... test)
- Between groups (control and test), are the completion rates (number of customers who reached the end of a step out of total who started) in each step the same? (... test)

### Completion Times
- Within the same group (control or test), are the completion times of all the steps of the process the same? (... test)
- Between groups (control and test), are the completion times of each step the same? (...step)

### Error Rates
- Is the error rate (number of customers who had to go to a previous step out of total) in the test group smaller than the error rate in the control group (whatever error rate is) - 5%?
    - No, it's just 1%.

### Segment Analysis for each rate analysis
- Do you find differences by age groups in control and test groups?

### Tableau
- Use Tableau to create graphs to summarize your findings.